In [1]:
# Dependencies
!pip install tiktoken
!pip install groq
!pip install langchain-groq
!pip install pinecone
!pip install langchain-community



Unable to create process using 'C:\Users\Ali Zain\computer programmer codes\New folder\python.exe "C:\Users\Ali Zain\computer programmer codes\New folder\Scripts\pip-script.py" install tiktoken'
Unable to create process using 'C:\Users\Ali Zain\computer programmer codes\New folder\python.exe "C:\Users\Ali Zain\computer programmer codes\New folder\Scripts\pip-script.py" install groq'
Unable to create process using 'C:\Users\Ali Zain\computer programmer codes\New folder\python.exe "C:\Users\Ali Zain\computer programmer codes\New folder\Scripts\pip-script.py" install langchain-groq'
Unable to create process using 'C:\Users\Ali Zain\computer programmer codes\New folder\python.exe "C:\Users\Ali Zain\computer programmer codes\New folder\Scripts\pip-script.py" install pinecone'
Unable to create process using 'C:\Users\Ali Zain\computer programmer codes\New folder\python.exe "C:\Users\Ali Zain\computer programmer codes\New folder\Scripts\pip-script.py" install langchain-community'


In [2]:
app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config["SQLALCHEMY_DATABASE_URI"] = 'mysql://root:@localhost/KSA_Stock_Project'
db = SQLAlchemy(app)

NameError: name 'Flask' is not defined

In [3]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
from sqlalchemy import func
from flask_sqlalchemy import SQLAlchemy
import spacy
import re
from flask import jsonify
from sqlalchemy import DECIMAL
import pymysql
from decimal import Decimal
import streamlit as st
import os
import json
import pathlib
import textwrap
from datetime import datetime
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
from datetime import date

from IPython.display import display
from IPython.display import Markdown

nlp = spacy.load("en_core_web_sm")
pymysql.install_as_MySQLdb()

In [4]:
app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config["SQLALCHEMY_DATABASE_URI"] = 'mysql+pymysql://root:@localhost/KSA_Stock_Project'
db = SQLAlchemy(app)

In [6]:
class User_info(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(120), nullable=False)
    Surname = db.Column(db.String(120), nullable=False)
    DOB_Day = db.Column(db.Integer, nullable=False)
    DOB_Month = db.Column(db.String(120), nullable=False)
    DOB_YEAR = db.Column(db.Integer, nullable=False)
    Gender = db.Column(db.String(120), nullable=False)
    Mobile_Number_Or_Email = db.Column(db.String(120), nullable=False)
    Password = db.Column(db.String(120), nullable=False)
    portfolios = db.relationship('Portfolio', backref='user', lazy=True)

class Stocks(db.Model):
    stock_id = db.Column(db.Integer, primary_key=True)
    ticker_symbol = db.Column(db.String(10), nullable=False)
    company_name = db.Column(db.String(100), nullable=False)
    current_price = db.Column(DECIMAL(10,2), nullable=False)
    sector = db.Column(db.String(50))
    listing_date = db.Column(db.Date)
    high_price_52w = db.Column(DECIMAL(10,2))
    low_price_52w = db.Column(DECIMAL(10,2))
    updated_at = db.Column(db.DateTime)
    portfolios = db.relationship('Portfolio', backref='stock', lazy=True)

class StockPriceHistory(db.Model):
    __tablename__ = 'stock price history'
    price_history_id = db.Column(db.Integer, primary_key=True)
    stock_id = db.Column(db.Integer, db.ForeignKey('stocks.stock_id'), nullable=False)
    price = db.Column(db.DECIMAL(10, 2), nullable=False)
    date = db.Column(db.TIMESTAMP, nullable=False, server_default=db.func.current_timestamp())

class Portfolio(db.Model):
    portfolio_id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user_info.id'), nullable=False)
    stock_id = db.Column(db.Integer, db.ForeignKey('stocks.stock_id'), nullable=False) 
    quantity = db.Column(db.Integer, nullable=False)
    purchase_price = db.Column(DECIMAL(10,2), nullable=False)
    purchase_date = db.Column(db.Date, nullable=False)

class Funds(db.Model):
    __tablename__ = 'funds'

    fund_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user_info.id'), nullable=False)
    available_balance = db.Column(db.Float, nullable=False, default=0.0)
    total_balance = db.Column(db.Float, nullable=False, default=0.0)
    last_updated = db.Column(db.DateTime, nullable=False)


c:\Users\Ali Zain\computer programmer codes\New folder\Lib\site-packages\flask_sqlalchemy\model.py:144: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User_info, and will be replaced in the string-lookup table.
  super().__init__(name, bases, d, **kwargs)


InvalidRequestError: Table 'user_info' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [7]:
# Import Libraries
import os
import pandas as pd
import tiktoken

from groq import Groq
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

from pinecone import Pinecone, ServerlessSpec
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

import io
from contextlib import redirect_stdout


In [8]:
groq_api_key = "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"

In [15]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import pandas as pd

# Connect to the database
app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config["SQLALCHEMY_DATABASE_URI"] = 'mysql+pymysql://root:@localhost/KSA_Stock_Project'
db = SQLAlchemy(app)

# Query the User_info table
with app.app_context():
    query = db.session.query(User_info).all()

    # Convert query results to a list of dictionaries
    data = [
        {
            'id': user.id,
            'name': user.name,
            'Surname': user.Surname,
            'DOB_Day': user.DOB_Day,
            'DOB_Month': user.DOB_Month,
            'DOB_YEAR': user.DOB_YEAR,
            'Gender': user.Gender,
            'Mobile_Number_Or_Email': user.Mobile_Number_Or_Email,
            'Password': user.Password
        }
        for user in query
    ]

    # Create a Pandas DataFrame
    imdb = pd.DataFrame(data)

    # Save the DataFrame to the CSV file
    imdb.to_csv('user_info.csv', index=False)
    print("Database information has been successfully added to the 'user_info.csv' file!")


Database information has been successfully added to the 'user_info.csv' file!


In [16]:
from sqlalchemy.orm import joinedload
import pandas as pd

# Perform the join
with app.app_context():
    query = (
        db.session.query(
            User_info.id.label("user_id"),
            User_info.name.label("user_name"),
            User_info.Surname.label("user_surname"),
            User_info.Gender.label("user_gender"),
            Stocks.stock_id.label("stock_id"),
            Stocks.ticker_symbol.label("ticker_symbol"),
            Stocks.company_name.label("company_name"),
            Stocks.current_price.label("current_price"),
            Stocks.sector.label("sector"),
            StockPriceHistory.price.label("stock_price"),
            StockPriceHistory.date.label("price_date"),
            Portfolio.quantity.label("quantity"),
            Portfolio.purchase_price.label("purchase_price"),
            Portfolio.purchase_date.label("purchase_date"),
            Funds.available_balance.label("available_balance"),
            Funds.total_balance.label("total_balance"),
            Funds.last_updated.label("fund_last_updated"),
        )
        .join(Portfolio, User_info.id == Portfolio.user_id)
        .join(Stocks, Portfolio.stock_id == Stocks.stock_id)
        .join(StockPriceHistory, Stocks.stock_id == StockPriceHistory.stock_id)
        .join(Funds, User_info.id == Funds.user_id)
    )

    # Fetch all results
    results = query.all()

    # Convert query results to a list of dictionaries
    data = [dict(row._mapping) for row in results]

    # Create a Pandas DataFrame
    combined_data = pd.DataFrame(data)

    # Save the combined data to a CSV file
    combined_data.to_csv('combined_data.csv', index=False)
    print("Combined data has been successfully saved to 'combined_data.csv'!")


Combined data has been successfully saved to 'combined_data.csv'!


In [20]:
# Create combined information column
combined_data['combined_info'] = combined_data.apply(
    lambda row: (
        f"User: {row['user_name']} {row['user_surname']} (Gender: {row['user_gender']}). "
        f"Stock: {row['ticker_symbol']} - {row['company_name']} in {row['sector']} sector, "
        f"current price: {row['current_price']}. "
        f"Portfolio: {row['quantity']} shares purchased at {row['purchase_price']} on {row['purchase_date']}. "
        f"Stock Price History: {row['stock_price']} on {row['price_date']}. "
        f"Funds: Available balance: {row['available_balance']}, Total balance: {row['total_balance']} (Last updated: {row['fund_last_updated']})."
    ),
    axis=1
)


In [18]:
# Token encoding
encoding = tiktoken.get_encoding('cl100k_base')
max_tokens = 8000

# Omit descriptions that are too long to embed
combined_data["n_tokens"] = combined_data.combined_info.apply(lambda x: len(encoding.encode(x)))
combined_data = combined_data[combined_data.n_tokens <= max_tokens]

# Use HuggingFace embeddings instead of OpenAI
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Generate embeddings
combined_data["vector"] = combined_data.combined_info.apply(lambda x: embeddings.embed_query(x))

In [21]:
# Get dimensions of Embeddings
embedding_dimensions = len(combined_data['vector'].iloc[0])
print(f"Embedding Dimensions: {embedding_dimensions}")

Embedding Dimensions: 384


In [22]:
combined_data.to_pickle('combined_data.pkl')

In [26]:
# Set the API key as an environment variable
os.environ["PINECONE_API_KEY"] = "pcsk_7YQanE_AP9y5db9N5vQaoUYC2h6bxvr92sEPyXzVBUcotcvBubtsEqsfkDaLQ6LrGwWRnw"

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [6]:


# Create or connect to a Pinecone index

embedding_dimensions = 384
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=embedding_dimensions,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"Index '{index_name}' already exists.")

# Prepare documents


# Create vector store with Pinecone



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'df2c386a891f1dfe27e8c55bb14c6c77', 'Date': 'Sat, 04 Jan 2025 05:16:28 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [38]:
documents = [
    Document(
        page_content=row['combined_info'],
        metadata={
            'user_id': row['user_id'],
        }
    ) for _, row in combined_data.iterrows()
]

In [39]:
index_name = "combineddataindex"
docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    index_name=index_name
)

In [40]:
# Define LLM using Groq (Llama model)
llm = ChatGroq(
    model_name="llama3-8b-8192",  # or another Llama model available
    api_key="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",
    temperature=0
)

In [42]:
# Define custom prompt template for stock-related information specific to the user
template = """You are a stock information assistant that helps users find details about stocks and their portfolios.
Use the following pieces of context to answer the question at the end.
If any information is missing from the database, provide a general answer based on your knowledge.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def retrieve_ranked_context(user_id, query):
    # Filter the documents based on user_id
    results = docsearch.similarity_search(query, k=100)  # Retrieve top 100 matches
    
    # Filter results to only include those related to the given user_id
    user_specific_results = [doc for doc in results if doc.metadata.get('user_id') == user_id]

    # Sort by relevant fields (e.g., stock price or portfolio details)
    ranked_results = sorted(
        user_specific_results,
        key=lambda x: float(x.metadata.get("price", 0)),  # Sort by stock price or other relevant field
        reverse=True
    )

    # If no results are found for the user, provide a general answer
    if not ranked_results:
        return "Sorry, I couldn't find any specific information for this user. Here's some general information about stocks and portfolios."

    # Create context from the top 3 results
    context = "\n".join([doc.page_content for doc in ranked_results[:3]])  # Top 3 results
    return context

# Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

def get_stock_info(user_id, query):
    # Suppress any print outputs during the execution
    with io.StringIO() as buf, redirect_stdout(buf):
        # Retrieve context specific to the user
        context = retrieve_ranked_context(user_id, query)
        qa_input = {"context": context, "query": query}
        result = qa_chain(qa_input)

    # Return the final result without any intermediate prints
    return result['result']


In [45]:
user_id = 1 # Example user_id
query =  "how many shares of Saudi Aramco i hold"  # Example query
result = get_stock_info(user_id, query)
print(result)

Based on the information provided, you hold 300 shares of Saudi Aramco (stock code 2222) in your portfolio.


In [ ]:
model.save('embedding_model')